In [1]:
from google.colab import files

# This will prompt you to upload a file
uploaded = files.upload()

Saving spam_ham_dataset.csv to spam_ham_dataset.csv


In [ ]:
import pandas as pd

In [2]:
import pandas as pd

# Function to try reading the file with different encodings
def read_file_with_encodings(filename):
    encodings = ['utf-8', 'latin1', 'iso-8859-1', 'cp1252']
    for encoding in encodings:
        try:
            df = pd.read_csv(filename, encoding=encoding)
            print(f'Successfully read the file with {encoding} encoding')
            return df
        except UnicodeDecodeError:
            print(f'Failed to read the file with {encoding} encoding')
    raise ValueError("Unable to read the file with available encodings")


# Modify the filename as necessary
data = read_file_with_encodings('spam_ham_dataset.csv')

# Display the first few rows of the dataset


Successfully read the file with utf-8 encoding


In [3]:
data.head()


,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [4]:
data=data[['label','text']]


In [5]:
data.head()

,label,text
0,ham,Subject: enron methanol ; meter # : 988291\r\n...
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see..."
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar..."
3,spam,"Subject: photoshop , windows , office . cheap ..."
4,ham,Subject: re : indian springs\r\nthis deal is t...


In [6]:
data.isnull().sum()

label    0
text     0
dtype: int64

In [7]:
data.duplicated().sum()

178

In [8]:
data=data.drop_duplicates()

In [9]:
data.duplicated().sum()

0

In [10]:
data=data.reset_index(drop=True)

In [11]:
data['label'].unique()

array(['ham', 'spam'], dtype=object)

In [12]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()


In [13]:
data['label'] = le.fit_transform(data['label'])


In [14]:
data['label'].unique()

array([0, 1])

In [15]:
import re
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [17]:
from nltk.corpus import stopwords


In [18]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()


In [19]:
cleaned_corpus = []
for i in range(0, len(data)):
    review = re.sub('[^a-zA-Z]', ' ', data['text'][i])
    review = review.lower()
    review = review.split()
    cleaned_corpus.append(review)


In [20]:
corpus = []
for review in cleaned_corpus:
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)


In [21]:
y=data['label']


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [23]:
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(corpus).toarray()


In [24]:
input_shape = X.shape[1]
print(f"The input layer should have {input_shape} neurons (features).")


The input layer should have 43162 neurons (features).


In [25]:
!pip install tensorflow


In [26]:
from sklearn.model_selection import train_test_split


In [27]:
X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.33, random_state=42)


In [28]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train, y_train)

In [29]:
y_pred=clf.predict(X_test)

In [30]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       1.00      0.90      0.94      1316
           1       0.71      0.99      0.83       332

    accuracy                           0.92      1648
   macro avg       0.85      0.94      0.89      1648
weighted avg       0.94      0.92      0.92      1648



In [ ]:
print(X_test.shape[1])
print(X_train.shape[1])

43162
43162


In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.layers import ReLU,ELU,Dropout
from tensorflow.keras.optimizers import Adam


In [32]:
model = Sequential()


In [33]:
model.add(Input(shape=(input_shape)))


In [34]:
model.add(Dense(30,activation='relu'))
model.add(Dropout(0.1))
# Hidden layer


In [35]:
model.add(Dense(10,activation='relu'))
# Hidden Layer

In [36]:
model.add(Dense(1,activation='sigmoid'))
# output

In [51]:
import tensorflow as tf
optimizer=Adam(learning_rate=1e-3)


In [52]:
model.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=['accuracy'])


In [53]:
from tensorflow.keras.callbacks import EarlyStopping


In [54]:
callback=tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    min_delta=0.0001,
    patience=20,
    verbose=1,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
)


In [55]:
model.fit(X_train, y_train, batch_size=10, epochs=1000,callbacks=callback)

Epoch 1/1000
335/335 [==============================] - 3s 4ms/step - loss: 1.8008e-07 - accuracy: 1.0000
Epoch 2/1000
335/335 [==============================] - 1s 4ms/step - loss: 1.2114e-07 - accuracy: 1.0000
Epoch 3/1000
335/335 [==============================] - 1s 4ms/step - loss: 5.9292e-08 - accuracy: 1.0000
Epoch 4/1000
335/335 [==============================] - 1s 4ms/step - loss: 2.9803e-08 - accuracy: 1.0000
Epoch 5/1000
335/335 [==============================] - 1s 4ms/step - loss: 2.0011e-08 - accuracy: 1.0000
Epoch 6/1000
335/335 [==============================] - 1s 4ms/step - loss: 1.8641e-08 - accuracy: 1.0000
Epoch 7/1000
335/335 [==============================] - 1s 4ms/step - loss: 9.1458e-09 - accuracy: 1.0000
Epoch 8/1000
335/335 [==============================] - 2s 5ms/step - loss: 7.4561e-09 - accuracy: 1.0000
Epoch 9/1000
335/335 [==============================] - 2s 5ms/step - loss: 1.2760e-08 - accuracy: 1.0000
Epoch 10/1000
335/335 [=======================

In [42]:
y_pred=model.predict(X_test)

52/52 [==============================] - 0s 4ms/step


In [43]:
y_pred=(y_pred>=0.5)

In [44]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[1169,   12],
       [   9,  458]])

In [45]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1181
           1       0.97      0.98      0.98       467

    accuracy                           0.99      1648
   macro avg       0.98      0.99      0.98      1648
weighted avg       0.99      0.99      0.99      1648

